In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  an

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [19]:
soup = BeautifulSoup(html, 'html5lib')
type(soup)

bs4.BeautifulSoup

In [5]:
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [36]:
_table = soup.find('table',{'class':'wikitable sortable'})
_rows = _table.find_all('tr')
# print(table_rows)

import csv

with open("editors.csv", "wt+", newline="") as f:
    writer = csv.writer(f)
    for row in _rows:
        csv_row = []
        for cell in row.findAll(["td", "th"]):
            csv_row.append(cell.get_text().strip())
        writer.writerow(csv_row)

In [37]:
df = pd.read_csv('editors.csv', error_bad_lines=False)
# df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [38]:
for idx, row in df.iterrows():
    if  df.loc[idx,'Borough'] == "Not assigned":
        df.drop(idx, axis=0, inplace= True)        
df.shape

(210, 3)

In [107]:
for idx, row in df.iterrows():
    if (df.loc[idx,'Neighbourhood'] == "Not assigned") & (df.loc[idx,'Borough'] == "Not assigned"):
        df.drop(idx, axis=0, inplace= True)    
    elif  df.loc[idx,'Neighbourhood'] == "Not assigned":
        df.loc[idx,'Neighbourhood'] = df.loc[idx,'Borough'] 

df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,Harbourfront,,
3,M6A,North York,Lawrence Heights,,
4,M6A,North York,Lawrence Manor,,
5,M7A,Downtown Toronto,Queen's Park,,
6,M9A,Queen's Park,Queen's Park,,
7,M1B,Scarborough,Rouge,,
8,M1B,Scarborough,Malvern,,
9,M3B,North York,Don Mills North,,


In [108]:
# df.reset_index(inplace=True)
df['Latitude']=""
df['Longitude']=""
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,Harbourfront,,
3,M6A,North York,Lawrence Heights,,
4,M6A,North York,Lawrence Manor,,
5,M7A,Downtown Toronto,Queen's Park,,
6,M9A,Queen's Park,Queen's Park,,
7,M1B,Scarborough,Rouge,,
8,M1B,Scarborough,Malvern,,
9,M3B,North York,Don Mills North,,


In [109]:
# df.drop('index', axis = 1, inplace = True)
!wget -q -O 'Geospatial.csv' https://cocl.us/Geospatial_data 
print('Data downloaded!')

Data downloaded!


In [110]:
df_csv = pd.read_csv('Geospatial.csv')
df_csv.set_index('Postal Code', inplace=True)
df_csv.head()
# df_csv.dtype

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [125]:
for idx, row in df.iterrows():
    psd = df.loc[idx,'Postcode']
    df.loc[idx, 'Latitude'] = df_csv.loc[psd,'Latitude']       
    df.loc[idx, 'Longitude'] = df_csv.loc[psd,'Longitude']

df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M6A,North York,Lawrence Heights,43.7185,-79.4648
4,M6A,North York,Lawrence Manor,43.7185,-79.4648


In [111]:
# Installing the packages needed to get the coordinates of our locations the points in our dataframe
# !conda install -c conda-forge geopy --yes 

# Installing the packages needed to display the points in our dataframe
# !conda install -c conda-forge folium=0.5.0 --yes 

import folium 
from geopy.geocoders import Nominatim 

address_Toronto = 'Toronto, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address_Toronto)
latitude = location.latitude
longitude = location.longitude

In [131]:
address_London = 'London, GB'

geolocator = Nominatim(user_agent="London_explorer")
location_London = geolocator.geocode(address_London)
latitude_London = location_London.latitude
longitude_London = location_London.longitude

In [126]:
# Create a new Dataframe for values containing Toronto
Toronto_df = df[df['Borough'].str.contains('Toronto') == True]
Toronto_df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
Latitude         object
Longitude        object
dtype: object

In [132]:
# Creating a new Dataframe from the list of boroughs in London
# Data used for this was gotten from https://raw.githubusercontent.com/reyjusuf/github-example/master/LondonCSV.csv
# In the case of this the coordinates for the boroughs is what was used to generate the map
  
# initialise data of lists. 
data = {'Borough':['City', 'Greenwich', 'Hackney', 'HammerSmith','Islington', 'Kesignton and Chealsea', 'Lambeth',
                  'Lewisham', 'Southwark', 'Tower Hamlets', 'Wandsworth', 'Westminster'], 
        'Latitude':[51.5074, 51.4934, 51.5734, 51.4912, 51.5465, 51.4991, 51.4571, 51.4415, 51.4834, 51.5203, 51.4571, 51.4975],
       'Longitude':[-0.1278,0.0098,-0.0724,-0.2237,-0.1058,-0.1938,-0.1231,-0.0117,-0.0821,-0.0293,-0.1818,-0.1357]
       } 
  
# Creates pandas DataFrame. 
df_London = pd.DataFrame(data) 
  
# print the data 
df_London.head()

,Borough,Latitude,Longitude
0,City,51.5074,-0.1278
1,Greenwich,51.4934,0.0098
2,Hackney,51.5734,-0.0724
3,HammerSmith,51.4912,-0.2237
4,Islington,51.5465,-0.1058


In [ ]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto



In [133]:
# create map of Toronto using latitude and longitude values
map_London = folium.Map(location=[latitude_London, longitude_London], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_London['Latitude'], df_London['Longitude'], df_London['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London